In [1]:
import numpy as np
import cv2 as cv
import os, sys, glob, scipy, argparse
import matplotlib.pyplot as plt
from scipy import signal
from scipy import fftpack, fft
from scipy.fftpack import fftshift, irfft, rfft 
from scipy.signal import blackman

ModuleNotFoundError: No module named 'cv2'

In [ ]:
"""
count = 1
#pathTest = '/Users/estepark/Documents/week1_thurs/GRR_Sample/Vermouth_EVT/20181126/Images/Reg/White/*.Png'
pathTest = '/Users/estepark/Documents/week1_thurs/GRR_Sample/Vermouth_EVT/20181126/Images/Reg/White'
isdir = os.path.isdir(pathTest) 
"""
class CrossHatch(object):
    """
    FUNCTION: 
    ** generate plots and pass/fail outputs based on frequency vs. distance 
    INPUTS: 
    ** path: the path that contains all the images
    ** rows: the number of rows to iterate thru
    ** col: the number of columns to iterate thru
    ?* filterType: crossHatch, faintLine, edgeGlow, luminanceHotspot, colorHotspot
    OUTPUTS:
    ** dict key: what # the image is in the dir
    ** dict values:  
    """
    def __init__(self, path , rows, col):
        assert os.path.isdir(pathTest), "hey"
        self.path = path
        self.rows = rows
        self.col = col
    def createDict(self):
        # define scharr kernal to convolve along diagonals
        kern = [[0, 1,  0], [1, 0, -1],[ 0, -1 ,0]]
        # define counter to number how many images in the dir
        count = 1
        white = {}
        img = [] # 1st value in dictionary
        imgScharr = [] # 2nd value in dictionary
        score = [] # will be used as numerical value > 0 that quantifies how many cols are crosshatched
        print("Image directory: " + str(self.path))
        print("Number of rows: " + str(self.rows))
        print("Number of col: " + str(self.col))
        for fname in glob.glob(self.path+'/*.Png'):
            #print(fname)
            img.append(cv.imread(fname)) #first value pair
            plt.figure()
            plt.title('White Image ' + str(count) + ": flipped from RGB --> BGR")
            plt.imshow(img[-1])
            #==============================================================
            # 1st VALUE, original pixel BGR mat 
            #==============================================================
            im = img.copy()
            imgray = cv.cvtColor(im[-1], cv.COLOR_BGR2GRAY)
            #print(imgray)
            
            #==============================================================
            # 2nd VALUE, Scharr pixel mat
            #==============================================================
            imgray = cv.bilateralFilter(imgray, 15, 75, 75)
            ret, thresh = cv.threshold(imgray, 10, 255, cv.THRESH_BINARY)
            contours, hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            im_with_contours = cv.drawContours(im[-1], contours, -1, (0,255,0), 6)
            diag_img = signal.convolve2d(imgray, np.array(kern), boundary='symm', mode='same')
            imgScharr.append(diag_img) #2nd value pair
            plt.figure()
            plt.title('White Image ' + str(count) + ": Scharr operator + Low pass filter")
            #plt.imshow(diag_img+101, cmap="gray")
            plt.imshow(diag_img , cmap="gray")
            plt.show()
            
            
            #================================================================
            # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html
            # https://docs.opencv.org/2.4/doc/tutorials/imgproc/imgtrans/sobel_derivatives/sobel_derivatives.html
            # ^sobelx vs. sobely, we see that sobely will perform better 
            # https://docs.opencv.org/trunk/da/d22/tutorial_py_canny.html
            # https://stackoverflow.com/questions/54950777/opencv-drawing-contours-with-various-methods-on-a-poor-image
            #================================================================
            plt.figure()
            diag_cpy = (((diag_img.copy()+2))/4*255).astype(np.uint8) #bc min=-2 and max=+2


            print('max = {}, min = {}'.format(np.max(diag_cpy), np.min(diag_cpy)))
            print(' diag shape = ',diag_cpy.shape)
            edged = cv.Canny(diag_cpy, 160, 230)
            plt.figure()
            plt.imshow(edged, cmap="gray")
            plt.show()
            
            sobelx = cv.Sobel(diag_cpy,cv.CV_64F,1,0,ksize=5)  
            sobely = cv.Sobel(diag_cpy,cv.CV_64F,0,1,ksize=5)
            print('sobel max = {}, min = {}'.format(np.max(sobely), np.min(sobely)))

            plt.figure()
            plt.imshow(sobelx)
            plt.show()
            plt.figure()
            plt.imshow(sobely)
            plt.show()
            cnts,_ = cv.findContours(edged.copy()[:,:50], cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
            print('contours shape {}'.format(np.array(cnts).shape))
            backtorgb = cv.cvtColor(diag_cpy,cv.COLOR_GRAY2RGB)
            cv.drawContours(backtorgb, cnts, -1, (0, 255, 0), 2)
            
            plt.imshow(backtorgb)
            plt.title('Edged Contours, Scharr -> canny')
            plt.show()
        
            
            #====================================================================================================
            # 3rd VALUE, 4 subplots: 1st) spatial domain BGR, 2) plot np.fft.fft, 3) stem fftpack.fftfreq
            #====================================================================================================
            samplingFreq = 100
            samplingInterval = 1 / samplingFreq
            beginTime = 0
            endTime = self.col
            teim = np.arange(beginTime,endTime, 1)
            countPeaks = [] # max valu = 646, counts how many stem peaks there are per column
            
            for i in range(self.col):
                #figure , axis = plt.subplots(3,1)
                freqRowSamps = np.linspace(0,self.rows-1, self.rows)
                # the issue is x-axis png ranges from 481, y-axis ranges 646, but these values change slightly based on image
                amplitude = imgray[:,i] # ampltiude == # of rows
                
                
                #print("--------------------------------------------------------")
                #plt.subplots_adjust ( hspace = 3)
                #time domain representation for CrossHatch periodic signal
                #axis[0].set_title('White Image #%s' % (count) +', Col #%s' % (i) + " Spatial Intensity vs. Pixel distance" )
                #axis[0].set_xlabel('Pixel Distance along y-axis ')
                #axis[0].set_ylabel('Pixel intensity')
                #axis[0].plot(amplitude)
                
                """
                # https://pythontic.com/visualization/signals/fouriertransform_fft
                plt.subplots_adjust ( hspace = 3)
                # frequency domain representation
                fourierTransf = np.fft.fft(amplitude)/ len(amplitude) # normalize amplitude
                fourierTransf = fourierTransf[range(int(len(amplitude)/2))] #exclude sampling freq??
                tpCount = len(amplitude)
                val = np.arange(int(tpCount/2))
                timePer = tpCount/samplingFreq
                freq = val/timePer 
                axis[1].set_title('White Image # %s' % (count) +', Col # %s' % (i) + " FFT Intensity vs. Pixel distance fft.fft plot " )
                axis[1].set_xlabel('Frequency along y-axis pixel locations')
                axis[1].set_ylabel('Amplitude')
                axis[1].plot(freq, abs(fourierTransf))
                # https://docs.scipy.org/doc/scipy-0.14.0/reference/tutorial/fftpack.html
                # Windowing the signal with a dedicated window function helps mitigate spectral leakage.
                # a Blackman window from scipy.signal and shows the effect of windowing
                # try irfft, rfft, fftshift?
                plt.subplots_adjust ( hspace = 3)
                yf = fft(amplitude)
                w = blackman(self.rows)
                ywf = irfft(w*yf)
                axis[3].set_title('White Image # %s' % (count) +', Col # %s' % (i) + " FFT Intensity vs. Pixel distance fft.fft semilogy" )
                axis[3].set_xlabel('Frequency along y-axis pixel locations')
                axis[3].set_ylabel('Amplitude')
                #axis[3].set_xlim(-samplingFreq / 2, samplingFreq / 2)
                #axis[3].set_ylim(-2, 100000)
                axis[3].semilogy(frequen, np.abs(ywf))
                print("--------------------------------------------------------")
                """
                
                # https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch04.html
                #plt.subplots_adjust ( hspace = 3)
                X = fftpack.fft(amplitude)
                amplitude1 = fftpack.fftfreq(len(amplitude))*samplingFreq
                #axis[1].set_title('Image #%s' % (count) +', Col #%s' % (i) + " FFT Intensity vs. Pixel distance (fftpack.fftfreq) stem " )
                #axis[1].set_xlabel('Frequency along y-axis pixel locations')
                #axis[1].set_ylabel('Amplitude')
                #axis[1].set_xlim(-5, samplingFreq / 1.2)
                #axis[1].set_ylim(-2, 600)
                #print(max(np.abs(X))) 
                #axis[1].stem(freqRowSamps, np.abs(X), use_line_collection = True) 
                
                #plt.subplots_adjust ( hspace = 3)     
                # https://pythonawesome.com/overview-of-the-peaks-dectection-algorithms-available-in-python/
                #freqPeaks = scipy.signal.find_peaks( np.abs(X) , height=[200, 1000000] )
                freqPeaks = scipy.signal.find_peaks( np.abs(X) , height=[200, 5000] )
                indexPeaks = freqPeaks[0]
                heightPeaks = np.array(list(freqPeaks[1].values()) , dtype = object)
                heightPeaks = heightPeaks[0]
                #print('height peaks: ' + str(heightPeaks))
                #print('index peaks: ' + str(indexPeaks))
                #axis[2].set_title('Image #%s' % (count) +', Col #%s' % (i) + " Peaks")
                #axis[2].set_ylabel('Amplitude')
                #axis[2].set_xlabel('Frequency along y-axis pixel locations')
                #axis[2].set_xlim(-5, samplingFreq / 1.2)
                #axis[2].set_ylim(-2, 600)
                if heightPeaks.size > 0:
                    #axis[2].stem(indexPeaks,heightPeaks, use_line_collection=True)
                    countPeaks.append(len(heightPeaks))
                else:
                    countPeaks.append(0)
            
            print('length crosshatch is ' + str(len(countPeaks)))
            isCrossHatch = sum(countPeaks)
            score.append(isCrossHatch)
            plt.figure()
            plt.plot(countPeaks,'bo')
            plt.title("Number of peaks in FFT spectrum vs. Column location")
            plt.xlabel("Column #")
            plt.ylabel("Number of peaks")
            plt.show()
            
            print( " img[count-1]" , img[count-1])
            print("imgScharr[count-1]," ,imgScharr[count-1])
            print("score[count-1])" ,score)
            print("score[count-1])" ,score[count-1])
            white.update( { 'white %s' % (count): (img[count-1]  , imgScharr[count-1], score[count-1])  } )

            count = count +1

In [ ]:
pathTest = '/Users/estepark/Documents/week1_thurs/GRR_Sample/Vermouth_EVT/20181126/Images/Reg/White'
isdir = os.path.isdir(pathTest) 

stuff = CrossHatch(path = pathTest, rows=  646, col= 480)
stuff.createDict()

In [ ]:
len(img)
count = 1
"""
objective: 
1) iterate through every 600 columns, find greatest variation in peaks

2) rank the greatest peak variations
"""
White = {}
for i in range(10):
    #print(count)
    White.update( { 'white %s' % (count): (img[i]  , imgScharr[i]) } )
    count = count +1 
    #print(White)

In [ ]:
import scipy
amplitude1 = [100,5,150,3,200,1,150,9,100] 
time = [1,2,3,4,5,6,7,8,9]
plt.plot(time,amplitude1)
a=scipy.signal.find_peaks(amplitude1,height=[125,175 ]  )
print("index is " + str(a[0]))
print(a[1].keys())
heights = np.array(list(a[1].values() ), dtype = float)
#heights = heights[0]
print("heights is " +str(heights))
print(len( heights  ))

In [ ]:
#labels =  '/Users/estepark/Documents/week9_tues/label'
annotations = '/Users/estepark/Documents/week11_mon/Vermouth_HVT/2018-09-10 D HVT HG8A/20180910/Data/HiResMesh/label'
originals = '/Users/estepark/Documents/week11_mon/Vermouth_HVT/2018-09-10 D HVT HG8A/20180910/Data/HiResMesh/results'

for fname in glob.glob(annotations+'/*.png'):
    serNum = fname.split('/')[-1][:-4] 
    
    annotation  = cv.imread(fname) # green line annotated label
    annotation_full = np.copy(annotation)
    annotation_binary = np.zeros(annotation.shape[:2])
    
    for i in range(annotation.shape[0]): 
        for j in range(annotation.shape[1])[::-1]:
            if annotation[i][j][0] == annotation[i][j][1] == annotation[i][j][2]:
                continue
            else:
                annotation_full[i][0:j][:] = annotation_full[i][j][:]
                annotation_binary[i][0:j] = 1
                break
    
    original = cv.imread(originals+'/{}.png'.format(serNum), cv.IMREAD_GRAYSCALE) # original label without annotation
    imgFilt = cv.bilateralFilter(original, 15, 75, 75)
    diag_img = signal.convolve2d(imgFilt, np.array(kern), boundary='symm', mode='same') # scharr filter 
    scharr = (((diag_img+2))/4*255).astype(np.uint8) #normalize after scharr, bc min=-2 and max=+2
    original = (original-np.min(original))/(np.max(original) - np.min(original)) # normalize label [0,1]

    
    plt.figure()
    plt.title(serNum+' label line')
    plt.imshow(annotation)
    plt.show()
    
    plt.figure()
    plt.title(serNum+' label full')
    plt.imshow(annotation_full)
    plt.show()
    
    plt.figure()
    plt.title(serNum+' label binary')
    plt.imshow(annotation_binary, cmap="gray")
    plt.show()
    
    plt.figure()
    plt.title(serNum+' label original')
    plt.imshow(original, cmap="gray")
    plt.show()
    
    plt.figure()
    plt.title(serNum+' scharr')
    plt.imshow(scharr, cmap="gray")
    plt.show()
    
    plt.figure()
    plt.title(serNum+' un scharr')
    plt.imshow(diag_img, cmap="gray")
    plt.show()